# Задача на 4-ом шаге урока

**🤜 Универсальный ужиматель 🤛**

В ноутбуке этого урока мы реализовали функцию reduce_mem_usage

In [1]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

🧠 Задача: Добавьте в функцию reduce_mem_usage возможность оптимизации категориальных признаков (названия этих колонок передаются в cat_cols в аргументах функции). Про оптимизацию хранения категориальных признаков мы рассказывали в ноутбуке к уроку.

In [2]:
# Добавьте в функцию reduce_mem_usage возможность оптимизации категориальных признаков
# (названия этих колонок передаются в cat_cols в аргументах функции). Про оптимизацию
# хранения категориальных признаков мы рассказывали в ноутбуке к уроку.

import numpy as np
import pandas as pd


def reduce_mem_usage(df, cat_cols=[]):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    # print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if col_type.startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            elif col_type.startswith('float'):
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif col in cat_cols:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    # print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    # print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

# Задача на 5-ом шаге урока

**Генераторы**

Напишите генератор, который принимает массив натуральных чисел и максимально возможное число в нем, а потом на каждой итерации возвращает OHE следующего элемента в массиве. Возвращаемый массив должен быть размерности (max_class + 1,).

In [3]:
import numpy as np

def ohe_generator(numbers, max_class):
    for num in numbers:
        row = np.zeros(max_class + 1, dtype=int)
        row[num] = 1
        yield row

# Задача на 6-ом шаге урока

In [4]:
import pandas as pd
import numpy as np

def get_df(i):
    # Создаём DataFrame с 1000 строк
    data = {"id": np.arange(i * 1000, (i + 1) * 1000),
            "value": np.random.random(1000),  # случайные значения от 0 до 1
            "category": np.random.choice(["A", "B", "C"], size=1000),  # случайные категории
            }
    return pd.DataFrame(data)

In [5]:
%%time
df = pd.DataFrame()
for i in range(1000):
    df = pd.concat([df, get_df(i)])

CPU times: total: 3.53 s
Wall time: 3.53 s


In [6]:
%%time
df = []
for i in range(1000):
    df.append(get_df(i))
df = pd.concat(df)

CPU times: total: 188 ms
Wall time: 180 ms


In [7]:
%%time
df = []
for i in range(1000):
    df += get_df(i).to_dict('records')
df = pd.DataFrame(df)

CPU times: total: 1.36 s
Wall time: 1.35 s


# Задача на 8-ом шаге урока

**🔎 Метод np.where()**

Датасет для тестирования

In [8]:
import pandas as pd
path = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_info.csv'
df_cars = pd.read_csv(path)
df_cars.sample(5)

,car_type,fuel_type,car_rating,year_to_start,riders,car_id,model,target_class,year_to_work,target_reg
633,economy,petrol,4.68,2014,57868,I-2609382N,Kia Rio X-line,engine_ignition,2015,55.906153
4211,economy,petrol,5.84,2013,46675,C-8802779C,Smart ForTwo,another_bug,2019,57.811777
4212,economy,petrol,2.72,2015,77676,f11409987C,Smart ForTwo,engine_check,2022,45.533980
845,economy,petrol,5.52,2012,18037,s-4094631m,VW Polo,break_bug,2016,50.807528
2615,economy,petrol,3.76,2016,102285,V12462480t,Smart Coupe,engine_check,2021,47.706855


Напишите функцию, которая возвращает car_id, если данная строка датафрейма удовлетворяет условию, иначе -1.

Условие: Либо year_to_start < 2015 и тип машины business, либо ее рейтинг строго больше 3, и ее модель - это одна из ['Hyundai Solaris', 'Smart ForFour', 'Renault Kaptur', 'Renault Sandero'].

In [9]:
models = {'Hyundai Solaris', 'Smart ForFour', 'Renault Kaptur', 'Renault Sandero'}

def get_where(df):
    mask1 = (df.year_to_start < 2015) & (df.car_type == 'business')
    mask2 = (df.car_rating > 3) & df.model.isin(models)
    return np.where(mask1 | mask2, df['car_id'], -1)

In [10]:
%%timeit
z = get_where(df_cars)

456 µs ± 3.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
models = np.array(['Hyundai Solaris', 'Smart ForFour', 'Renault Kaptur', 'Renault Sandero'])

def get_where(df):
    # Векторизованные условия
    mask1 = (df['year_to_start'].values < 2015) & (df['car_type'].values == 'business')
    mask2 = (df['car_rating'].values > 3) & np.isin(df['model'].values, models)
    # Возвращаем результат через np.where
    return np.where(mask1 | mask2, df['car_id'].values, -1)

In [12]:
%%timeit
z = get_where(df_cars)

240 µs ± 798 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Задача на 10-ом шаге урока

**🍢 Векторизация**

Дана функция

In [13]:
def f(x):
    if 'Audi' == x['model']:
        if x['car_rating'] > 3:
            return 0
        else:
            if x['fuel_type'] == 'petrol':
                return 1
            return np.nan
    elif x['year_to_start'] in [2015, 2016, 2017] or x['car_rating'] > 4:
        return round(x['car_rating'] - 4.5, 3) * 10
    else:
        return 3

In [14]:
import pandas as pd

path = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_info.csv'
df_cars = pd.read_csv(path)

необходимо объявить векторизованную версию этой функции и запишите ее в переменную vectfunc. Для тестирования используется код:

Решение

In [15]:
import numpy as np

def func(car_rating, model, year_to_start, fuel_type):
    if model == 'Audi':
        if car_rating > 3:
            return 0
        elif fuel_type == 'petrol':
            return 1
        return np.nan
    elif year_to_start in {2015, 2016, 2017} or car_rating > 4:
        return round(car_rating - 4.5, 3) * 10
    return 3


vectfunc = np.vectorize(func)

In [16]:
preds = vectfunc(df_cars['car_rating'],
                 df_cars['model'],
                 df_cars['year_to_start'],
                 df_cars['fuel_type'])

# Задача на 11-ом шаге урока

**🚦 Mетод np.select()**

У вас есть датафрейм df_cars

In [17]:
import pandas as pd
import numpy as np


path = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_info.csv'
df_cars = pd.read_csv(path)

Напишите conditions, choices и default_value, которые нужно будет использовать в np.select(), чтобы посчитать следующую функцию:

In [18]:
def f(x):
    if 'Audi' == x['model']:
        if x['car_rating'] > 3:
            return 0
        else:
            if x['fuel_type'] == 'petrol':
                return 1
            return np.nan
    elif x['year_to_start'] in [2015, 2016, 2017] or x['car_rating'] > 4:
        return round(x['car_rating'] - 4.5, 3) * 10
    else:
        return 3

Для проверки будет использоваться следующий код:

Решение

In [19]:
import numpy as np

conditions = [(df_cars['model'] == 'Audi') & (df_cars['car_rating'] < 3),
              (df_cars['model'] == 'Audi') & (df_cars['fuel_type'] == 'petrol'),
              (df_cars['model'] == 'Audi'),
              df_cars['year_to_start'].isin({2015, 2016, 2017}) | (df_cars['car_rating'] > 4)
              ]
choices = [0, 1, np.nan, round(df_cars.car_rating - 4.5, 3) * 10]
default_value = 3

In [20]:
preds = np.select(conditions, choices, default=default_value)

In [21]:
# V2
conditions = [df_cars.model.__eq__('Audi') & df_cars.car_rating.__lt__(3),
              df_cars.model.__eq__('Audi') & df_cars.fuel_type.__eq__('petrol'),
              df_cars.model.__eq__('Audi'),
              df_cars.year_to_start.isin({2015, 2016, 2017}) | df_cars.car_rating.__gt__(4)
              ]
choices = [0, 1, np.nan, round(df_cars.car_rating - 4.5, 3) * 10]
default_value = 3

preds2 = np.select(conditions, choices, default=default_value)

print(np.all(preds == preds2))

True


# Задача на 13-ом шаге урока

In [22]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

path = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_info.csv'
df_cars = pd.read_csv(path)

lbl = preprocessing.LabelEncoder()
df_cars['int_model'] = lbl.fit_transform((df_cars['model'] + df_cars['fuel_type']).astype(str))

У вас есть датафрейм df_cars, в котором есть два столбца: int_model и target_reg. 

Задача: Написать функцию, которая для каждой группы (по int_model) будет считать среднее, сумму, минимум и кол-во элементов в группе. 

Для подсчета данных значений не используйте pd.groupby(), а используйте векторизацию через numpy.

Решение

In [23]:
def fast_groupby(df):
    indices = df['int_model'].values
    target_reg = df['target_reg'].values
    count_values = np.bincount(indices)
    sum_values = np.bincount(indices, weights=target_reg)
    min_values = np.minimum.reduceat(target_reg[np.argsort(indices)],
                                     np.concatenate(([0],
                                                     np.cumsum(np.bincount(indices))))[:-1])
    mean_values = sum_values / count_values
    res = pd.DataFrame({'int_model': np.arange(indices.max() + 1),
                        'min': min_values,
                        'sum': sum_values,
                        'count': count_values,
                        'mean': mean_values})
    return res

In [24]:
%%timeit
df_cars.groupby('int_model', as_index=False)['target_reg'].agg(min='min',
                                                               sum='sum',
                                                               count='count',
                                                               mean='mean',
                                                              )

575 µs ± 2.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
%%timeit
fast_groupby(df_cars)

207 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
import polars as pl

# Преобразование в Polars DataFrame
df_cars_pl = pl.DataFrame(df_cars)

In [27]:
%%timeit
df_cars_pl.group_by('int_model').agg([
    pl.col('target_reg').min().alias('min'),
    pl.col('target_reg').sum().alias('sum'),
    pl.col('target_reg').count().alias('count'),
    pl.col('target_reg').mean().alias('mean')
])

151 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
grp = df_cars.groupby('int_model', as_index=False)['target_reg'].agg(min='min',
                                                                     sum='sum',
                                                                     count='count',
                                                                     mean='mean',
                                                                     )
grp2 = fast_groupby(df_cars)

print((grp.round(10) == grp2.round(10)).all())

int_model    True
min          True
sum          True
count        True
mean         True
dtype: bool


In [29]:
grp3 = df_cars_pl.group_by('int_model').agg([
    pl.col('target_reg').min().alias('min'),
    pl.col('target_reg').sum().alias('sum'),
    pl.col('target_reg').count().alias('count'),
    pl.col('target_reg').mean().alias('mean')
])
grp3 = grp3.to_pandas().sort_values('int_model').reset_index(drop=True)
print((grp.round(10) == grp3.round(10)).all())

int_model    True
min          True
sum          True
count        True
mean         True
dtype: bool


In [30]:
# V2
# Вместо np.argsort использовал np.unique для нахождения уникальных значений в столбце
# int_model, которые представляют собой различные группы, по которым будут проводиться
# агрегационные операции (сумма, среднее значение, минимум, и количество элементов).
# В этом контексте np.unique выполняет функцию аналогичную .groupby() в Pandas, но только для
# того чтобы определить, какие уникальные группы (int_model в данном случае) существуют в
# данных. Это нужно для последующей векторизации операций на этих группах. Опция
# return_index=True возвращает индексы первого вхождения каждого уникального значения в
# каждой группе.
# В функции np.bincount использовал параметр minlength, который определяет минимальную длину
# выходного массива. Этот параметр используется для того, чтобы удостовериться, что выходной
# массив имеет определенную длину, даже если действительные значения индексов (или их
# количество) меньше этой длины. Значения для индексов, которые не появляются во входных
# данных, будут заполнены нулями.
# В нашем случае, minlength=len(unique_models) гарантирует, что массивы count_values и
# sum_values имеют одинаковую длину и соответствуют числу уникальных моделей (unique_models).
# Это упрощает последующие вычисления и помогает избежать ошибок, связанных с несоответствием
# размеров массивов.
# Этот параметр особенно полезен, когда в данных есть "пропуски" в уникальных значениях.
# Например, если int_models содержит [0, 1, 2, 4, 5], но не содержит 3, np.bincount с
# параметром minlength все равно вернет массив с нулем для индекса 3, что соответствует
# отсутствующим значениям.
# Для расчета минимума, аналогично автору, использовал функцию np.minimum.reduceat, которая
# выполняет уменьшающую операцию с использованием функции np.minimum на разделах массива
# targets на основе индексов в массиве indices.
# targets: это массив значений, которые нужно уменьшить, значения из столбца target_reg в
# отсортированном порядке;
# indices: это массив индексов, указывающих начало каждой новой группы в массиве targets. .
# Теперь, что делает np.minimum.reduceat:
# Он берет подмассив из targets, начиная с первого индекса из indices и до следующего индекса
# в indices (не включая его), и находит минимальное значение в этом подмассиве.
# Затем повторяет эту операцию для всех подмассивов, определенных индексами в indices.
# Возвращает массив минимальных значений для каждой группы.
# Простой пример кода:
# import numpy as np
# targets = np.array([5, 2, 7, 1, 3, 8, 9, 10])
# indices = np.array([0, 3, 5])
# result = np.minimum.reduceat(targets, indices)
# print(result)  # Вывод: [2 1 8]
# Индексы в массиве indices указывают на начальные позиции подмассивов в массиве targets,
# которые нужно рассмотреть. В данном примере:
# indices = np.array([0, 3, 5])
# а так эти индексы разбивают targets на подмассивы:
# первая группа начинается с индекса 0 и идет до индекса 3 (не включительно): [5, 2, 7]
# вторая группа начинается с индекса 3 и идет до индекса 5 (не включительно): [1, 3]
# третья группа начинается с индекса 5 и идет до конца массива (поскольку нет последующего
# индекса после 5 в массиве indices): [8, 9, 10]
# Именно так работает метод np.minimum.reduceat. Он использует массив indices для разделения
# targets на подмассивы и вычисления минимума для каждого подмассива.
# Авторская реализация вычисляет минимальные значения для каждой группы, используя также для
# начала сортировку и кумулятивные суммы, которые являются индексами, указывающими на начало
# каждой группы:
# сначала сортируются значения target_reg в соответствии с отсортированными индексами
# int_model
# sorted_ind = df_cars['target_reg'].values[np.argsort(indices)]
# далее, автор вычисляет позиции, на которых начинаются новые группы
# concated = np.concatenate(([0], np.cumsum(np.bincount(indices))))[:-1]
# минимальные значения для каждой группы вычисляются с помощью
# np.minimum.reduceat(sorted_ind, concated) применяющий операцию "минимум" к подмассивам
# массива sorted_ind, разделенным согласно индексам в массиве concated
# 'min': np.minimum.reduceat(sorted_ind, concated),
# Таким образом, np.minimum.reduceat эффективно заменяет цикл поиска минимального значения
# для каждой группы, что является частью операции groupby в pandas.

def fast_groupby(df_cars):
    # Сортируем DataFrame по int_model для последующей векторизации
    df_sorted = df_cars.sort_values(by='int_model')
    
    # Извлекаем значения из отсортированного DataFrame
    int_models = df_sorted['int_model'].values
    targets = df_sorted['target_reg'].values

    # Находим индексы, где начинаются новые группы
    unique_models, indices = np.unique(int_models, return_index=True)

    # Подсчет количества, суммы и минимума для каждой группы
    count_values = np.bincount(int_models, minlength=len(unique_models))
    sum_values = np.bincount(int_models, weights=targets, minlength=len(unique_models))
    min_values = np.minimum.reduceat(targets, indices)

    # Подсчет среднего значения для каждой группы
    mean_values = sum_values / count_values

    # Создание результирующего DataFrame
    res = pd.DataFrame({'int_model': unique_models,
                        'min': min_values,
                        'sum': sum_values,
                        'count': count_values,
                        'mean': mean_values
                        })

    return res

In [31]:
%%timeit
fast_groupby(df_cars)

453 µs ± 3.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [32]:
grp4 = fast_groupby(df_cars)

print((grp.round(10) == grp4.round(10)).all())

int_model    True
min          True
sum          True
count        True
mean         True
dtype: bool
